In [4]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import numpy as np
import tensorboard
import optuna

In [5]:
def make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight, healthy_reward):
    """
    Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium con i parametri specificati.
    """
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make("Ant-v5", 
                    reset_noise_scale=reset_noise_scale, 
                    forward_reward_weight=forward_reward_weight, 
                    ctrl_cost_weight=ctrl_cost_weight, 
                    healthy_reward=healthy_reward, 
                    render_mode='None')

In [3]:
# Hyperparameter tuning con Optuna

def objective(trial):
    # Definisci lo spazio di ricerca degli iperparametri
    reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.0, 1.0)
    forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.0, 1.0)
    ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.0, 1.0)
    healthy_reward = trial.suggest_float('healthy_reward', 0.0, 1.0)
    
    # Crea l'ambiente con gli iperparametri suggeriti
    env = make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight, healthy_reward)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)
    
    # Definisci altri iperparametri del modello
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    n_steps = trial.suggest_int('n_steps', 2048, 8192, step=2048)
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])
    gamma = trial.suggest_float('gamma', 0.9, 0.9999)
    gae_lambda = trial.suggest_float('gae_lambda', 0.8, 1.0)
    clip_range = trial.suggest_float('clip_range', 0.1, 0.3)
    ent_coef = trial.suggest_float('ent_coef', 0.0, 0.1)
    
    # Crea ed allena il modello PPO
    model = PPO("MlpPolicy", env, learning_rate=learning_rate, n_steps=n_steps, batch_size=batch_size, gamma=gamma, gae_lambda=gae_lambda, clip_range=clip_range, ent_coef=ent_coef, verbose=0)
    model.learn(total_timesteps=200000)
    
    # Valuta il modello
    episodes = 200
    episode_rewards = []
    for episode in range(episodes):
        obs = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action, _states = model.predict(obs)
            obs, reward, done, info = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)
    
    # Ritorna la ricompensa media come obiettivo
    return np.mean(episode_rewards)

# Crea uno studio Optuna e ottimizza l'obiettivo
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

# Stampa i migliori iperparametri trovati
print("Best hyperparameters: ", study.best_params)

[I 2025-02-10 11:22:18,395] A new study created in memory with name: no-name-9978e25d-6599-4b08-a775-895197237ada
/var/folders/5w/qb_kxxjs5lscg9_8tpttrzs40000gn/T/ipykernel_5085/1608488940.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[I 2025-02-10 11:23:37,081] Trial 0 finished with value: 1.1700634956359863 and parameters: {'reset_noise_scale': 0.404410847191928, 'forward_reward_weight': 0.3198649582694849, 'ctrl_cost_weight': 0.055607876577841164, 'healthy_reward': 0.877856969284288, 'learning_rate': 0.0006372045914501103, 'n_steps': 4096, 'batch_size': 128, 'gamma': 0.9104759636688363, 'gae_lambda': 0.9686818090920738, 'clip_range': 0.28345078373963734, 'ent_coef': 0.08973498221344439}. Best is trial 0 with value: 1.1700634956359863.
[I 2025-02-10 